<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [0]:
# Import any packages you need here
import copy 


#Now, define the class PuzzleNode:


class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    
    '''
    Attributes of PuzzleNode Class: 
      state: the node's current configuration of the 8 piece puzzle
      path_length: The minimum number of moves required to reach the current node
      f_value: Evaluation function return value for current state
      parent: Parent node 
      pruned: Whether or not another node with same state a path costis available, which renders current node irrelevant in A* search 
    '''
    
    def __init__(self, state, path_length, fval, parent): 
      self.state = state
      self.path_length = path_length
      self.f_value = fval
      self.parent = parent
      self.pruned = False 
      
     
    '''
    Overload operator:
      A function that orders nodes compared to other nodes also placed in a priority queue, based on f values
    '''
    def __lt__(self, other):
      return self.f_value < other.f_value
      
    '''
    Generate Children:
      Creates all possible child states from current state, within the rules of 8-puzzle game 
    
    '''
    def generate_children(self):
      
      #Find the coordinate of the zero in current state 
      subarray_with_zero =  [x for x in self.state if 0 in x][0]
      zero_x_coordinate, zero_y_coordinate = self.state.index(subarray_with_zero), subarray_with_zero.index(0)
      
      #Initialize return children array
      children = []
      
      #For each possible direction in which the zero can move, if it is a legal move, 
      #move the zero and append resulting state as a child to the children array
      for dx,dy in [[-1,0],[1,0],[0,1],[0,-1]]: 
        new_state = copy.deepcopy(self.state)
        if 0 <= zero_x_coordinate + dx < len(self.state) and 0 <= zero_y_coordinate + dy < len(self.state[0]): 
          temp = new_state[zero_x_coordinate][zero_y_coordinate]
          new_state[zero_x_coordinate][zero_y_coordinate] = new_state[zero_x_coordinate + dx][zero_y_coordinate + dy]
          new_state[zero_x_coordinate + dx][zero_y_coordinate + dy] = temp 
          children.append(new_state)
      return children 
    '''
    __str__() function 
      Function to print out the grid 
    '''
    def __str__(self): 
      for row in self.state: 
        print(row)
    '''
    Get Goal Function: 
      Gets the final goal state where every tile is in the perfect position. 
    '''
    
    def get_goal(self): 
      return_state = copy.deepcopy(self.state)
      counter = 0 
      for i in range(len(self.state)): 
        for j in range(len(self.state[0])):
          return_state[i][j] = counter
          counter += 1
      return return_state
   

In [36]:
'''
PuzzleNode Unit Tests
'''

#Define a 8 Puzzle state 
a = [[1,0,2],[3,4,5],[6,7,8]]

#Create a Puzzle Node Object 
s1 = PuzzleNode(a,0,0,None)

#Attribute Values Test
print("State: " + str(s1.state))
print("Path Length: " + str(s1.path_length))
print("F Value: " + str(s1.path_length))
print("Parent: " + str(s1.parent))
print("Pruned: " + str(s1.pruned))

#Methods Test
print("Print State as string")
s1.__str__()
print("Children: " ,s1.generate_children())
print("Goal State: " ,s1.get_goal())

State: [[1, 0, 2], [3, 4, 5], [6, 7, 8]]
Path Length: 0
F Value: 0
Parent: None
Pruned: False
Print State as string
[1, 0, 2]
[3, 4, 5]
[6, 7, 8]
Children:  [[[1, 4, 2], [3, 0, 5], [6, 7, 8]], [[1, 2, 0], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]
Goal State:  [[0, 1, 2], [3, 4, 5], [6, 7, 8]]


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [0]:
# Add any additional code here (e.g. for the memoization extension)

# Misplaced tiles heuristic
def misplaced_tiles(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    Method: 
      A counter iterates through the 2d array, increasing it's value by 1 at 
      each value. If the value encountered in the state is not equal to the 
      value of the counter, then the value is incorrect
    """
    misplaced_tiles_count = 0
    counter = 0
    
    for i in range(len(state)): 
      for j in range(len(state[0])): 
        if counter != 0 and state[i][j] != counter: 
          misplaced_tiles_count += 1
        counter += 1 
    return misplaced_tiles_count 

# Manhattan distance heuristic
def manhattan_distance(state):
    """
    This function returns the Manhattan distance from the solved state, given 
    the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    Method: 
      First, a completed state is generated. Then, for each tile in the current
      state, the distance in x and y values from the goal state is calculated and 
      summed, giving the total Manhattan distance. 
    """
    counter = 0
    rows, columns = len(state), len(state[0])
    manhattan_distance = 0 
    
    #Create completed state 
    completed_state = [[0]*columns for row in range(rows)]
    for i in range(len(state)): 
      for j in range(len(state[0])): 
        
        completed_state[i][j] = counter
        counter += 1 
        
    for i in range(len(state)): 
      for j in range(len(state[0])): 
        value = state[i][j]
        if value != 0: 
          #Find the indices of the value in the completed state 
          subarray_with_value =  [x for x in completed_state if value in x][0]
          value_completed_x_coordinate, value_completed_y_coordinate = completed_state.index(subarray_with_value), subarray_with_value.index(value)
          
          #Find the manhattan distance between the current and goal state for tile
          addition = abs(value_completed_x_coordinate - i) + abs(value_completed_y_coordinate - j)
          
          #Add manhattan distance of tile to the total manhattan distance 
          manhattan_distance += addition
    
    #return manhattan distance 
    return manhattan_distance
    


def euclidean_distance(state): 
    """
    This function returns the euclidean distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the euclidean distance from the solved configuration
    Method: 
      A Similar method is followed to the manhattan distance, but instead of 
      summing the horizontal and vertical distances, the diagonal distance 
      is found using the pythagoran theorem. 
    """
    counter = 0
    rows, columns = len(state), len(state[0])
    euclidean_distance = 0 
    
    #Create completed state 
    completed_state = [[0]*columns for row in range(rows)]
    for i in range(len(state)): 
      for j in range(len(state[0])): 
        
        completed_state[i][j] = counter
        counter += 1 
        
    for i in range(len(state)): 
      for j in range(len(state[0])): 
        value = state[i][j]
        if value != 0: 
          subarray_with_value =  [x for x in completed_state if value in x][0]
          value_completed_x_coordinate, value_completed_y_coordinate = completed_state.index(subarray_with_value), subarray_with_value.index(value)
          addition = ((abs(value_completed_x_coordinate - i))**2 + (abs(value_completed_y_coordinate - j))**2)**0.5
          euclidean_distance += addition
        
    return euclidean_distance
 
  
def linear_conflict_distance(state):
  
  '''
  A linear conflict is defined as the following: 
  "Two tiles j and k are in a linear conflict if tj and tk are in the same 
  line, the goal positions of j and k are both in that line, j is to the 
  right of k and goal position of j is to the left of the goal position of tk."
  
  This is found by first finding the number of linear conflicts along the rows
  of the state, then flipping the state and finding the number of conflicts
  which corresponds to the number of conflicts in the columns of the original
  state 
  
  '''
  
  total_linear_conflicts = 0 
  
  
  #Create completed state 
  
  rows, columns = len(state), len(state[0])
  completed_state = [[0]*columns for row in range(rows)]
  counter = 0
  for i in range(len(state)): 
    for j in range(len(state[0])): 
      completed_state[i][j] = counter
      counter += 1 
  
  #Function to flip the state 
  def flip_state(state): 
    for i in range(0, len(state)):
      for j in range(i+1, len(state)):
          state[i][j],state[j][i] = state[j][i],state[i][j]
    return state
  
  #Find the number of linear conflicts 
  def linear_conflicts(state,completed):
    linear_conflict_count = 0
    for i in range(3):
      counter = 0
      temp = []
      for j in range(0,3): 
        if state[i][j] in completed[i]: 
          temp.append(state[i][j])
          counter += 1 

      if counter == 2: 
        A1 = completed_state[i].index(temp[0])
        A2 = completed_state[i].index(temp[1])
        B1 = state[i].index(temp[0])
        B2 = state[i].index(temp[1])
        if (A1-A2>0 and B1-B2<0) or (A1-A2<0 and B1-B2>0): 
          linear_conflict_count += 1 
      if counter == 3: 
        A1 = completed_state[i].index(temp[0])
        A2 = completed_state[i].index(temp[1])
        A3 = completed_state[i].index(temp[2])

        B1 = state[i].index(temp[0])
        B2 = state[i].index(temp[1])
        B3 = state[i].index(temp[2])

        if (A1-A2>0 and B1-B2<0) or (A1-A2<0 and B1-B2>0):
          linear_conflict_count += 1
        if (A1-A3>0 and B1-B3<0) or (A1-A3<0 and B1-B3>0):
          linear_conflict_count += 1
        if(A3-A2>0 and B3-B2<0) or (A3-A2<0 and B3-B2>0): 
          linear_conflict_count += 1 
          
    return linear_conflict_count
  flipped_state = flip_state(state)
  
  total_linear_conflicts = linear_conflicts(state,completed_state) + linear_conflicts(flipped_state,completed_state)
  
  return manhattan_distance(state) + total_linear_conflicts*2
    
   
      
        

heuristics = [misplaced_tiles, manhattan_distance, euclidean_distance]

In [0]:
#Unit tests for heuristics, copied from tests provided by Assignment

## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [0]:
# Import any packages or define any helper functions you need here
from queue import PriorityQueue
import copy 

def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        state: The initial state of the puzzle as a list of lists
        heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        exp: The number of nodes expanded to reach the solution
        max_frontier: The maximum size of the frontier over the whole search
        opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    
    '''
    Part 1 - test for state being correctly defined. 
    '''
    
    #If the state is not of n*n dimensions, return err = -1
    for i in range(len(state)):
      if len(state) != len(state[i]): 
        steps, exp, max_frontier = 0,0,0
        err = -1 
        opt_path = None
        return steps, exp, max_frontier,opt_path, err
    
    #If the state does not have all elements from 0 to n-1, return err = -1 
    #Method - create a array of all values, remove values from state, if unable,
    #         state is invalid.  
    all_elements_array = [i for i in range(len(state)**2)]
    for i in range(len(state)): 
      for j in range(len(state[0])): 
        value = state[i][j]
        if value in all_elements_array: 
          all_elements_array.remove(value)
        else: 
          steps, exp, max_frontier = 0,0,0
          err = -1 
          opt_path = None
          return steps, exp, max_frontier,opt_path, err
        
    '''
    Part 2 - Initialize return variables, starting node, variables for A* search, A* search 
    '''
    
    '''
    General outline of A* search for 8-Puzzle
    
    Until goal state is reached: 
    
      Get the node that has the lowest f val available => named current_node
      
      Get all the children of current node being evaluated. 
      
      If any child had previously been initialized, compare path costs, ignore
      child with the higher path cost
      
      Append children to priority queue 
    
    This is a working A* search algorithm because the implementation of a 
    priority queue for search makes it a Best-First-Search algorithm where
    the nodes with a minimum (estimated) cost are expanded first. 
    
    It is an improvement over the typical A* search algorithm because of it's 
    use of a dictionary for costs:db, which enables the algorithm as a whole
    to have a O(n) runtime, and a O(1) lookup time for each instance of 
    checking if a child node has been previously initialized and stored in
    costs_db. This was done through a unique use of typecasting the state
    as a tuple of tuples, which allowed it to be hashed and stored in a 
    dictionary
    
    EXTENSION:
    
    1)
    The Algorithm returns the goal state and the number of step required to 
    reach it, if it can be reached, as it is complete. Every possible state
    will be explored until the goal reach is reached or all possible nodes are
    exhausted. Therefore, if the goal state cannot be reached, there 
    will be no nodes left to expand, the priority queue used, frontier, will be 
    empty and the last node explored, current_node, will have a state that is 
    not equal to the goal state. This is checked for and if it is the case, 
    a return value of -2 is returned. 
    
    2)
    2 different heuristics were created - Euclidean distance and conflicting tiles. 
    Unfortunately, neither were able to outperform Manhattan distances, despite 
    getting the same distances. 
    
    3)
    Memoization occurs because when a node was is created from any state, 
    the a tuple of the state and the node is stored in the costs_db dictionary. 
    The state is hashed and used as a key and the node itself, including it's 
    f_value attribute, is stored as the value to the key. Then, when the same
    state is created again, if it exists in the dictionary, the fval is simply
    retrieved from the previous insatiation of the node for that state, instead
    of being re-calculated. 
    '''
    '''
    
      Return variables
        max_frontier is a return value for the maximum size of the frontier 
        expansion_counter is a counter of total number of states explored 
        counter is a counter of number of steps from initial to goal state 

        starting_node is a node creating from the starting state. 
        
        steps counts the number of steps from initial to goal state
        
        optimal_path is the optimal path of states from initial to goal state
     '''

    #Return values 
    max_frontier = 0 
    expansion_counter = 0     
    steps = 0
    optimal_path = []
    
    '''
      Starting node
        state is given
        
        path cost is 0 initially
        
        f value is calculated by adding g value(path cost) to heuristic calculation of state
        
        No parent is present
     '''
    #Create a starting Node 
    path_cost = 0 
    starting_node = PuzzleNode(state, path_length = 0, fval = heuristic(state)+path_cost ,parent = None)
    
    '''
      Variables for A* search 
        goal_state is obtained to create exit condition for the loop
        
        costs_db keeps track of the path lengths of nodes, and is updated to 
          keep it at a minimum
        
        frontier is a priority queue of nodes, ordered by their f values. It 
          aids in the implementation of the A* search algorithm, which is a
          Best-First-Search algorithm, as the node with the lowest f_value 
          is put at the back of the priority queue, and is expanded on first. 
          This is a result of the __lt__() standard operator function in the 
          PuzzleNode() class 
    
    '''
    #Create variables for A* Search
    goal_state = starting_node.get_goal() 
    costs_db = {tuple(map(tuple, starting_node.state)):starting_node} 
    frontier = PriorityQueue()
    frontier.put(starting_node)
    
    
    
    
    #If frontier is empty, that means that all possible moves from the starting 
    #   state have been explored
    while not frontier.empty(): 
      
      #The node that is the most optimal in terms of f_values is selected. 
      #If it is the goal node, is it returned. If it is pruned, it is ignored
      current_node = frontier.get() 
      if current_node.state == goal_state: break
      if current_node.pruned: continue
 
      
      #All possible children states are created. If there has been previous 
      #  initiations of these children, the child that has the smaller path
      #  length is kept and the rest are ignored. 
      children = current_node.generate_children() 
    
      for child in children: 
        #A tuple version of the child state is created in order to allow the use
        #  of a dictionary data structure, which reduces lookup time from O(n^2)
        #  to O(1)
        child_tuple = tuple(map(tuple, child))
        path_length = current_node.path_length + 1 
        
        if child_tuple in costs_db: 
          #Memoization is implemented here, as explained above.f_value is never 
          #  calculated twice, only once.If it was previously calculated, the 
          #  previous value is used. 
          
          hval = costs_db[child_tuple].f_value - costs_db[child_tuple].path_length
          fval = hval + path_length
          
          if costs_db[child_tuple].path_length > path_length:
            costs_db[child_tuple].pruned = True
          else:
            #The continue function skips to the next iteration of the loop.
            continue 
        else: 
          fval = heuristic(child) + path_length 
        
        #A new child node is created and put in frontier as well as the dictionary
        next_node = PuzzleNode(child,path_length,fval,current_node) 
        frontier.put(next_node)  
        costs_db[child_tuple] = next_node
        
        #Expansion counter increased by 1 as 1 additional node has been expanded
        expansion_counter = expansion_counter + 1
        

      #max frontier updated 
      max_frontier = max(max_frontier, len(frontier.queue))
    
    
    
    
    #Error value updated based on whether a goal state was reached or not. 
    if current_node.state != goal_state: 
      err = -2
    else: 
      err = 0 
    
    
    #Optimal path is recreated, and steps counted based on number of transition
    # states between initial and goal state through the optimal path. 
    optimal_path = [current_node.state]
    while current_node.parent:
      steps += 1 
      optimal_path.append(current_node.parent.state)
      current_node = current_node.parent
    #optimal path is reversed to display in order from start to end
    optimal_path = optimal_path[::-1]
    
              
    #Final value is returned. 
    return steps, expansion_counter, max_frontier, optimal_path, err

          
    

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [0]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert (err == -1)

In [0]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [0]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]

for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [0]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [0]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [45]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    print("DOM:",dom)
assert(dom > 0)

DOM: -61
DOM: -3322
DOM: -9408


AssertionError: ignored

In [0]:
## Memoization test - will be carried out after submission


#Completed, do refer to comments in solvePuzzle function

###References

Jason, Y. (n.d.). Linear Distance Calculator . Retrieved October 26, 2019, from https://github.com/Jason-Yuan/8PuzzleGameSovler/blob/master/heuristic_estimate.py.Stack Overflow . (2011, March 1). 

Rotating a two-dimensional array in Python. Retrieved October 26, 2019, from https://stackoverflow.com/questions/8421337/rotating-a-two-dimensional-array-in-python.Yar Kan, A. (2016, May 3). 

Implementing A-star(A*) to solve N-Puzzle. Retrieved from https://algorithmsinsight.wordpress.com/graph-theory-2/a-star-in-general/implementing-a-star-to-solve-n-puzzle/.
